In [4]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import tabulate
from collections import  namedtuple

In [5]:
df = pd.read_csv("data.csv", index_col=0, delimiter=";")
df

,Fakulta,AQ
#,,
1,LF,20
5,FF,30
7,FF,19
11,FTK,8
15,PedF,6
...,...,...
977,PrF,12
986,CMTF,30
988,PrF,12


In [6]:
df.describe()

,AQ
count,300.000000
mean,17.960000
std,6.932431
min,5.000000
25%,12.000000
50%,17.000000
75%,22.250000
max,35.000000


In [ ]:
from autoviz.AutoViz_Class import AutoViz_Class
%matplotlib inline
AV = AutoViz_Class()
dfte = AV.AutoViz("data.csv", sep=";")

Statistical tests scipy: https://docs.scipy.org/doc/scipy/reference/stats.html#statistical-tests


# Úkol 10: ANOVA
V minulosti jsme se již seznámili s teorií empatizace a systematizace Simona Barona-Cohena. Z jeho poznatků by mělo vyplývat to, že jedinci, u kterých se ve větší míře vyskytují (subklinické) autistické rysy, by měli být více přitahovaní exaktními, teoretickými obory a technickými obory. Oproti tomu, jedinci s nižší mírou těchto rysů, budou orientování spíše na druhé lidi a humanitní obory.

Rozsáhlé skupině studentů UPOL jsme administrovali dotazník subklinického autismu (AQ). U studentů bylo mimoto sledováno, jakou navštěvují fakultu. U AQ předpokládejme normální rozdělení.

Před začátkem testování vyřaďte respondenty, kteří neuvedli, jakou fakultu navštěvují. Dále, pokud zjistíte, že některá fakulta je zastoupena méně než 10 studenty, tak tyto studenty z analýzy vyřaďte a s jejich fakultou nepočítejte.

Pozn.: U Tukeyho testu v tomto cvičení nepoužívejte adaptaci pro nestejné skupiny (Spjotvoll & Stoline, 1973), kterou program nabízí.

=====================================================

In [4]:
df.groupby("Fakulta", dropna=False).size()

Fakulta
CMTF     7
FF      97
FTK     12
FZV      6
LF      24
PF      22
PedF    36
PrF     75
NaN     21
dtype: int64

In [5]:
df_filter = df
# Remove subjects not assigned to any Faculty
df_filter = df_filter.loc[~df_filter["Fakulta"].isna()]

# Remove faculties with nr responses < 10 
df_filter = df_filter.loc[~df_filter["Fakulta"].isin(["CMTF", "FZV"])]

df_filter.groupby("Fakulta", dropna=False).agg(["size", "mean", "std"])


AQ                     
        size       mean       std
Fakulta                          
FF        97  18.515464  6.788645
FTK       12  13.750000  7.225271
LF        24  16.083333  7.198530
PF        22  16.363636  6.035967
PedF      36  17.444444  6.652366
PrF       75  19.786667  6.996705

Pomocí ANOVy ověřte, zdali průměrné výsledek v AQ souvisí s fakultou, kterou jedinec navštěvuje. Uveďte nalezenou hodnotu statistiky F.

In [6]:
group_names = list(df_filter["Fakulta"].unique())
group_samples_list = []
for group_name in group_names:
    group_samples = list(df_filter.query(f"Fakulta == '{group_name}'")["AQ"])
    group_samples_list.append(group_samples)

group_samples_list

stats.f_oneway(*group_samples_list)


F_onewayResult(statistic=2.729355061073364, pvalue=0.020091309136077862)

(Skripta, str. 118)

Scheffého testem prozkoumejte rozdíly mezi skupinami. Jaké hodnotě se rovná nejnižší pozorovaná p-hodnota?

In [7]:
# Scheffe's test is not available in scipy, 
# using test from scikit_posthocs instead
import scikit_posthocs

results = scikit_posthocs.posthoc_scheffe(df_filter, val_col="AQ", group_col="Fakulta")
results


,LF,FF,FTK,PedF,PrF,PF
LF,1.000000,0.785110,0.967446,0.989135,0.377812,0.999997
FF,0.785110,1.000000,0.394572,0.985669,0.916540,0.878098
FTK,0.967446,0.394572,1.000000,0.755962,0.155958,0.950387
PedF,0.989135,0.985669,0.755962,1.000000,0.721288,0.996739
PrF,0.377812,0.916540,0.155958,0.721288,1.000000,0.512222
PF,0.999997,0.878098,0.950387,0.996739,0.512222,1.000000


In [8]:
np.min(results.values)

0.15595757187325832

Tukeyho HSD testem prozkoumejte rozdíly mez skupinami. Jaké hodnotě se rovná nejnižší pozorovaná p-hodnota?

In [9]:
print(group_names)

['LF', 'FF', 'FTK', 'PedF', 'PrF', 'PF']


In [10]:
results = stats.tukey_hsd(*group_samples_list)
results.statistic
results.pvalue

array([[1.        , 0.62428872, 0.92813904, 0.97440022, 0.19286639,
        0.9999927 ],
       [0.62428872, 1.        , 0.20591183, 0.96668823, 0.83148492,
        0.76568714],
       [0.92813904, 0.20591183, 1.        , 0.5841658 , 0.05399431,
        0.8942538 ],
       [0.97440022, 0.96668823, 0.5841658 , 1.        , 0.53867653,
        0.99197469],
       [0.19286639, 0.83148492, 0.05399431, 0.53867653, 1.        ,
        0.30786109],
       [0.9999927 , 0.76568714, 0.8942538 , 0.99197469, 0.30786109,
        1.        ]])

In [11]:
np.min(results.pvalue)

0.0539943062379179

Pomocí Leveneova testu ověřte hypotézu o existenci rozdílů v rozptylech skórů AQ u jednotlivých skupin z předešlého bodu. Uveďte nalezenou hodnotu statistiky F.

In [12]:
stats.levene(*group_samples_list, center="mean")

LeveneResult(statistic=0.07912492909500432, pvalue=0.995395459395527)

Pomocí Welchova testu ověřte, zdali průměrný výsledek v AQ souvisí s fakultou, kterou jedinec navštěvuje. Uveďte F statistiku.

In [13]:
def welch_anova_np(*args, var_equal=False):
    # https://svn.r-project.org/R/trunk/src/library/stats/R/oneway.test.R
    # translated from R Welch ANOVA (not assuming equal variance)

    F_onewayResult = namedtuple('F_onewayResult', ('statistic', 'pvalue'))

    args = [np.asarray(arg, dtype=float) for arg in args]
    k = len(args)
    ni =np.array([len(arg) for arg in args])
    mi =np.array([np.mean(arg) for arg in args])
    vi =np.array([np.var(arg,ddof=1) for arg in args])
    wi = ni/vi

    tmp =sum((1-wi/sum(wi))**2 / (ni-1))
    tmp /= (k**2 -1)

    dfbn = k - 1
    dfwn = 1 / (3 * tmp)

    m = sum(mi*wi) / sum(wi)
    f = sum(wi * (mi - m)**2) /((dfbn) * (1 + 2 * (dfbn - 1) * tmp))
    prob = stats.f.sf(dfbn, dfwn, f)   # equivalent to stats.f.sf
    return F_onewayResult(f, prob)

welch_anova_np(*group_samples_list)

F_onewayResult(statistic=2.51501828979903, pvalue=0.13499216983826456)